# STEP 1


## Permutations

### Problem

Generate all permutations of a list of distinct numbers.
Example input: `[1, 2, 3]`.

### Logic

Use backtracking: build the permutation incrementally by choosing an unused element at each position. Track a `path` and a `used` set (or swap in-place). When `path` length == n, record a copy.

### Step-by-step trace (input `[1,2,3]`)

We show the recursion choices (path) and remaining elements:

1. Start: `path = []`, remaining `{1,2,3}`
2. Choose 1 → `path = [1]`, remaining `{2,3}`

   * Choose 2 → `path = [1,2]`, remaining `{3}`

     * Choose 3 → `path = [1,2,3]` → output `[1,2,3]`
   * Backtrack to `path = [1]`
   * Choose 3 → `path = [1,3]`, remaining `{2}`

     * Choose 2 → `path = [1,3,2]` → output `[1,3,2]`
3. Backtrack to `path = []`
4. Choose 2 → `path = [2]`, remaining `{1,3}`

   * ... yields `[2,1,3]`, `[2,3,1]`
5. Choose 3 → `path = [3]`, remaining `{1,2}`

   * ... yields `[3,1,2]`, `[3,2,1]`

### Python code

```python
def permutations(nums):
    res = []
    n = len(nums)
    used = [False]*n
    path = []

    def backtrack():
        if len(path) == n:
            res.append(path.copy())
            return
        for i in range(n):
            if used[i]:
                continue
            used[i] = True
            path.append(nums[i])
            backtrack()
            path.pop()
            used[i] = False

    backtrack()
    return res

## Example
print(permutations([1,2,3]))
```

### Output

```
[[1, 2, 3],
 [1, 3, 2],
 [2, 1, 3],
 [2, 3, 1],
 [3, 1, 2],
 [3, 2, 1]]
```

---

## Combinations (n choose k)

### Problem

Generate all k-length combinations from a list of distinct numbers.
Example input: `nums = [1,2,3,4]`, `k = 2`.

### Logic

Backtracking with start index to avoid duplicates/permutations. At each step pick an element from `start` to end, append it, recurse with `start = i+1`. Stop when `path` length == k.

### Step-by-step trace (`[1,2,3,4]`, k=2)

1. Start: `path = []`, start=0
2. Choose 1 (i=0) → `path=[1]`, start=1

   * Choose 2 (i=1) → `path=[1,2]` → output `[1,2]`
   * Backtrack → `path=[1]`
   * Choose 3 → output `[1,3]`
   * Choose 4 → output `[1,4]`
3. Backtrack → `path=[]`
4. Choose 2 (i=1) → `path=[2]`

   * Choose 3 → `[2,3]`
   * Choose 4 → `[2,4]`
5. Choose 3 (i=2) → `path=[3]`

   * Choose 4 → `[3,4]`

### Python code

```python
def combinations(nums, k):
    res = []
    path = []
    n = len(nums)

    def backtrack(start):
        if len(path) == k:
            res.append(path.copy())
            return
        for i in range(start, n):
            path.append(nums[i])
            backtrack(i+1)
            path.pop()

    backtrack(0)
    return res

## Example
print(combinations([1,2,3,4], 2))
```

### Output

```
[[1, 2],
 [1, 3],
 [1, 4],
 [2, 3],
 [2, 4],
 [3, 4]]
```

---

## Subsets (Power set)

### Problem

Generate all subsets (the power set) of a list of distinct numbers.
Example input: `[1,2,3]`.

### Logic

Backtracking or recursion: at each index decide to include `nums[i]` or not. This yields a binary recursion tree. Use `start` index and append current `path` at every node (if you want all subsets) or only when hitting end.

### Step-by-step trace (`[1,2,3]`)

We show include/exclude decisions:

1. Start `path=[]`, i=0

   * Include 1 → `path=[1]`, i=1

     * Include 2 → `path=[1,2]`, i=2

       * Include 3 → `[1,2,3]`
       * Exclude 3 → `[1,2]`
     * Exclude 2 → `path=[1]`

       * Include 3 → `[1,3]`
       * Exclude 3 → `[1]`
   * Exclude 1 → `path=[]`

     * Include 2 → `path=[2]` ...
       Produces: `[], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]` (ordering depends on implementation)

### Python code

```python
def subsets(nums):
    res = []
    path = []
    n = len(nums)

    def backtrack(start):
        ## record current subset (copy)
        res.append(path.copy())
        for i in range(start, n):
            path.append(nums[i])
            backtrack(i+1)
            path.pop()

    backtrack(0)
    return res

## Example
print(subsets([1,2,3]))
```

### Output

```
[[],
 [1],
 [1, 2],
 [1, 2, 3],
 [1, 3],
 [2],
 [2, 3],
 [3]]
```

---

## N-Queens

### Problem

Place `n` queens on an `n x n` chessboard so that no two queens attack each other. Return all distinct solutions. Each solution is commonly shown as a list of strings where `'Q'` is a queen and `'.'` is empty. Example: `n = 4`.

### Logic

Backtracking by rows: place one queen per row at a column that isn't attacked. Track three sets (or boolean arrays) for columns, `diag1` (row+col), and `diag2` (row-col) to test attacks in O(1). When row == n, we have a solution.

### Step-by-step trace (n=4) — high level

We attempt row-wise placements:

1. row0: try col0, col1, col2, col3 — each choice leads to recursive attempts for row1.
2. After exploring, two solutions are found for n=4:

   * Solution A:

     ```
     . Q . .
     . . . Q
     Q . . .
     . . Q .
     ```
   * Solution B:

     ```
     . . Q .
     Q . . .
     . . . Q
     . Q . .
     ```

(Trace is large; this is the compact high-level path that yields the two valid boards.)

### Python code

```python
def solve_n_queens(n):
    res = []
    cols = [False]*n
    diag1 = [False]*(2*n)   ## r + c
    diag2 = [False]*(2*n)   ## r - c + n

    board = [['.']*n for _ in range(n)]

    def backtrack(row):
        if row == n:
            res.append([''.join(r) for r in board])
            return
        for c in range(n):
            if cols[c] or diag1[row+c] or diag2[row-c+n]:
                continue
            cols[c] = diag1[row+c] = diag2[row-c+n] = True
            board[row][c] = 'Q'
            backtrack(row+1)
            board[row][c] = '.'
            cols[c] = diag1[row+c] = diag2[row-c+n] = False

    backtrack(0)
    return res

## Example
solutions = solve_n_queens(4)
for sol in solutions:
    for row in sol:
        print(row)
    print()
```

### Output (for n=4)

```
.Q..
...Q
Q...
..Q.

..Q.
Q...
...Q
.Q..
```

---

If you want, I can:

* Add inline visual recursion trees for any of these (ASCII-style).
* Provide versions that handle duplicates (e.g., permutations with repeated elements).
* Bundle all functions into a single Python module with tests and timing.

Which follow-up would you like? 😄


# Step 2

Perfect 👌 — this is another **core DSA pattern** every interview expects you to master deeply.
Let’s cover **Binary Search Pattern** with examples in your requested format:

👉 **problem → logic → step-by-step trace → Python code → output**

We'll cover three classic cases under this pattern:

1. **Basic Binary Search (Search in sorted array)**
2. **Search in Rotated Sorted Array**
3. **Search with Monotonic Condition (Peak Element / Minimize-Maximize problems)**

---

### 🧩 1. Basic Binary Search — *Search in Sorted Array*

#### 🧠 Problem

Given a sorted array `nums` and a target `x`, return the index of `x` if found, else `-1`.

Example:

```
nums = [1, 3, 5, 7, 9, 11]
target = 7
```

---

#### ⚙️ Logic

Binary search works by **halving the search space** each time:

1. Find mid = (low + high)//2
2. If nums[mid] == target → found
3. If target < nums[mid] → search left half
4. Else → search right half
5. Continue until low > high

Time complexity: **O(log n)**

---

#### 🧭 Step-by-step Trace

| Step | low | high | mid | nums[mid] | Compare with target | Action                       |
| ---- | --- | ---- | --- | --------- | ------------------- | ---------------------------- |
| 1    | 0   | 5    | 2   | 5         | 5 < 7               | Move right (`low = mid + 1`) |
| 2    | 3   | 5    | 4   | 9         | 9 > 7               | Move left (`high = mid - 1`) |
| 3    | 3   | 3    | 3   | 7         | 7 == 7              | Found at index 3             |

---

#### 💻 Python Code

```python
def binary_search(nums, target):
    low, high = 0, len(nums) - 1
    while low <= high:
        mid = (low + high) // 2
        if nums[mid] == target:
            return mid
        elif target < nums[mid]:
            high = mid - 1
        else:
            low = mid + 1
    return -1

## Example
print(binary_search([1,3,5,7,9,11], 7))
```

---

#### 🧾 Output

```
3
```

---

### 🔄 2. Search in Rotated Sorted Array

#### 🧠 Problem

You’re given an array originally sorted but rotated at an unknown pivot, e.g.
`nums = [4,5,6,7,0,1,2]`, find `target = 0`.

---

#### ⚙️ Logic

Even though rotated, **one half is always sorted**.
Steps:

1. Find mid.
2. If nums[mid] == target → found.
3. If left half is sorted (`nums[low] <= nums[mid]`):

   * Check if target in that range (`nums[low] <= target < nums[mid]`) → move left.
   * Else → move right.
4. Else (right half sorted):

   * Check if target in that range (`nums[mid] < target <= nums[high]`) → move right.
   * Else → move left.

---

#### 🧭 Step-by-step Trace

Array: `[4,5,6,7,0,1,2]`, target = 0

| Step | low | high | mid | nums[mid] | Sorted half | Action                      |
| ---- | --- | ---- | --- | --------- | ----------- | --------------------------- |
| 1    | 0   | 6    | 3   | 7         | Left sorted | 0 not in [4,7) → move right |
| 2    | 4   | 6    | 5   | 1         | Left sorted | 0 in [0,1) → move left      |
| 3    | 4   | 4    | 4   | 0         | Found!      |                             |

---

#### 💻 Python Code

```python
def search_rotated(nums, target):
    low, high = 0, len(nums) - 1
    while low <= high:
        mid = (low + high) // 2
        if nums[mid] == target:
            return mid

        ## Left half sorted
        if nums[low] <= nums[mid]:
            if nums[low] <= target < nums[mid]:
                high = mid - 1
            else:
                low = mid + 1
        else:
            ## Right half sorted
            if nums[mid] < target <= nums[high]:
                low = mid + 1
            else:
                high = mid - 1
    return -1

## Example
print(search_rotated([4,5,6,7,0,1,2], 0))
```

---

#### 🧾 Output

```
4
```

---

### 📈 3. Monotonic Condition — *Find Peak Element (or Binary Search on Answer)*

#### 🧠 Problem

Given an array where adjacent elements are **different**, find an index `i` such that
`nums[i]` is a **peak** (greater than both neighbors).
For simplicity, consider boundaries as `-∞`.

Example:

```
nums = [1,2,1,3,5,6,4]
```

Possible peaks: `2` (index 1) and `6` (index 5). Return any.

---

#### ⚙️ Logic

Observe that the array has a **monotonic property**:

* If `nums[mid] < nums[mid + 1]` → peak lies **right side**.
* Else → peak lies **left side** (including mid).

Keep halving until you find the peak.

---

#### 🧭 Step-by-step Trace

| Step | low | high | mid | nums[mid] | nums[mid+1] | Move                                  |
| ---- | --- | ---- | --- | --------- | ----------- | ------------------------------------- |
| 1    | 0   | 6    | 3   | 3         | 5           | Right (since 3 < 5)                   |
| 2    | 4   | 6    | 5   | 6         | 4           | Left (since 6 > 4)                    |
| 3    | 4   | 5    | 4   | 5         | 6           | Right → low=5 → Peak found at index 5 |

---

#### 💻 Python Code

```python
def find_peak(nums):
    low, high = 0, len(nums) - 1
    while low < high:
        mid = (low + high) // 2
        if nums[mid] < nums[mid + 1]:
            low = mid + 1
        else:
            high = mid
    return low  ## or high, both point to peak

## Example
nums = [1,2,1,3,5,6,4]
print(find_peak(nums))
```

---

#### 🧾 Output

```
5
```

👉 `nums[5] = 6` is a peak.

---

### 🧭 Pattern Summary

| Type                 | Key Condition / Trick                   | Example                                         |
| -------------------- | --------------------------------------- | ----------------------------------------------- |
| **Basic Search**     | `nums[mid] == target`                   | `[1,3,5,7,9]`                                   |
| **Rotated Array**    | Identify **sorted half** first          | `[4,5,6,7,0,1,2]`                               |
| **Monotonic Search** | Move where the function still increases | Peak Element / Binary Search on Answer problems |

---

Would you like me to continue this **Binary Search Pattern** with advanced “Binary Search on Answer” problems (like **Minimum days to ship packages**, **Koko eating bananas**, **Allocate books**)?
They use the *same monotonic reasoning* but on value ranges instead of indices — very popular in interviews.


Excellent 🔥 — you’ve now reached the **Top-K / Heap pattern**, one of the *must-master patterns* in DSA for coding interviews (Amazon, Google, etc.).

Let’s break it carefully with your requested approach:
👉 **problem → logic → step-by-step trace → Python code → output**

We’ll go through **3 core problems** covering the most asked heap concepts:

1. **Kth Largest Element in Array**
2. **Top K Frequent Elements (Frequency Sort)**
3. **Sliding Window Maximum**

---

### 🥇 1. Kth Largest Element in an Array

#### 🧠 Problem

Given an array of integers `nums` and an integer `k`, return the **kth largest element**.

Example:
`nums = [3,2,1,5,6,4]`, `k = 2` → Output: `5`

---

#### ⚙️ Logic

Use a **min-heap of size k**:

1. Iterate through the array.
2. Push each element into the heap.
3. If heap size > k, pop the smallest.
4. The top (root) of the heap will be the kth largest.

⏱️ **Time complexity:** O(n log k)

---

#### 🧭 Step-by-step Trace

| Step                     | Element | Heap (min-heap)          | Action      |
| ------------------------ | ------- | ------------------------ | ----------- |
| 1                        | 3       | [3]                      | push        |
| 2                        | 2       | [2,3]                    | push        |
| 3                        | 1       | [1,3,2] → pop(1) → [2,3] | heap size>k |
| 4                        | 5       | [2,3,5] → pop(2) → [3,5] |             |
| 5                        | 6       | [3,5,6] → pop(3) → [5,6] |             |
| 6                        | 4       | [4,6,5] → pop(4) → [5,6] |             |
| ✅ Result → `heap[0] = 5` |         |                          |             |

---

#### 💻 Python Code

```python
import heapq

def find_kth_largest(nums, k):
    heap = []
    for n in nums:
        heapq.heappush(heap, n)
        if len(heap) > k:
            heapq.heappop(heap)
    return heap[0]

## Example
print(find_kth_largest([3,2,1,5,6,4], 2))
```

---

#### 🧾 Output

```
5
```

---

### 🥈 2. Top K Frequent Elements (Frequency Sort)

#### 🧠 Problem

Given an array of integers `nums`, return the `k` most frequent elements.
Example:
`nums = [1,1,1,2,2,3], k = 2` → Output: `[1,2]`

---

#### ⚙️ Logic

1. Count frequency of each number using `Counter`.
2. Use a **min-heap of size k** storing `(frequency, element)`.
3. Push items into the heap; if heap size exceeds k, pop smallest frequency.
4. Extract elements from heap.

⏱️ **Time complexity:** O(n log k)

---

#### 🧭 Step-by-step Trace

| Step                                     | Element | Freq | Heap                                      | Action            |
| ---------------------------------------- | ------- | ---- | ----------------------------------------- | ----------------- |
| 1                                        | 1       | 3    | [(3,1)]                                   | push              |
| 2                                        | 2       | 2    | [(2,2),(3,1)]                             | push              |
| 3                                        | 3       | 1    | push → pop (since size>k) → [(2,2),(3,1)] | pop smallest freq |
| ✅ Heap contains top 2 frequent → `[1,2]` |         |      |                                           |                   |

---

#### 💻 Python Code

```python
from collections import Counter
import heapq

def top_k_frequent(nums, k):
    freq = Counter(nums)
    heap = []
    for num, count in freq.items():
        heapq.heappush(heap, (count, num))
        if len(heap) > k:
            heapq.heappop(heap)
    return [num for count, num in heap]

## Example
print(top_k_frequent([1,1,1,2,2,3], 2))
```

---

#### 🧾 Output

```
[2, 1]
```

(*Order may vary depending on heap structure*)

---

### 🥉 3. Sliding Window Maximum

#### 🧠 Problem

Given an integer array `nums` and a window size `k`, return the **maximum** of each window as it slides from left to right.

Example:
`nums = [1,3,-1,-3,5,3,6,7]`, `k = 3`
→ Output: `[3,3,5,5,6,7]`

---

#### ⚙️ Logic

Two main approaches:

* **Heap-based (O(n log k))** → maintain a max heap with valid indices.
* **Deque-based (O(n)) optimal)** → maintain decreasing order of elements in deque.

We’ll use the **heap-based** one here since we’re focusing on **heap pattern**.

Steps:

1. Push `(-num, index)` to simulate a max heap.
2. For each index:

   * Pop from heap if index is outside the window.
   * Heap’s top gives max of current window.

---

#### 🧭 Step-by-step Trace

`nums = [1,3,-1,-3,5,3,6,7]`, k=3

| i                         | num                | Heap (value,index)                   | Window    | Max |
| ------------------------- | ------------------ | ------------------------------------ | --------- | --- |
| 0                         | 1                  | [(-1,0)]                             | [1]       | —   |
| 1                         | 3                  | [(-3,1),(-1,0)]                      | [1,3]     | —   |
| 2                         | -1                 | [(-3,1),(-1,0), (1,2)]               | [1,3,-1]  | 3   |
| 3                         | -3                 | [(-3,1),(-1,0), (1,2), (3,3)]        | [3,-1,-3] | 3   |
| 4                         | 5                  | push(-5,4) → pop old indices → max=5 |           |     |
| …                         | continue sliding … | …                                    | …         | …   |
| ✅ Result: `[3,3,5,5,6,7]` |                    |                                      |           |     |

---

#### 💻 Python Code

```python
import heapq

def max_sliding_window(nums, k):
    heap, result = [], []
    for i, num in enumerate(nums):
        ## push negative for max-heap
        heapq.heappush(heap, (-num, i))
        ## Remove out-of-window elements
        while heap[0][1] <= i - k:
            heapq.heappop(heap)
        if i >= k - 1:
            result.append(-heap[0][0])
    return result

## Example
print(max_sliding_window([1,3,-1,-3,5,3,6,7], 3))
```

---

#### 🧾 Output

```
[3, 3, 5, 5, 6, 7]
```

---

### ⚡ Pattern Summary

| Problem Type           | Heap Type         | Key Idea               | Time                           |
| ---------------------- | ----------------- | ---------------------- | ------------------------------ |
| **Kth Largest**        | Min-heap (size k) | Keep top k largest     | O(n log k)                     |
| **Top K Frequent**     | Min-heap (size k) | Count frequency + heap | O(n log k)                     |
| **Sliding Window Max** | Max-heap          | Maintain window max    | O(n log k) or O(n) using deque |

---

Would you like me to extend this next to
➡️ **“Two Heaps Pattern”** (Median of Data Stream, Sliding Median, Meeting Rooms II) — it’s the natural continuation of this Top-K section.
